## C - Loading, Saving and Freezing Embeddings

> Trong notebook này, chúng ta se cùng tìm hiểu các để load custom word embedding trong TorchText, cách để save tất cả các embeddings ta học được trong lúc train và cách để freeze/unfreeze embeddings trong lúc train.

### Loading Custom Embeddings

> Để có thể load được, custom embedding phải có cấu trúc như sau: mỗi dòng gồm một từ luôn đúng ở đầu câu, tiếp đến là các giá trị của embedding vector của từ đó, tất cả ngăn cách nhau bởi space. Tất cả các vector phải có cùng kich thước.

> Dưới đây là một ví dụ về format mà custom embedding cần tuân theo:

In [1]:
with open('custom_embeddings/embeddings.txt', 'r') as f:
    print(f.read())

good 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
great 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
awesome 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
bad -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0
terrible -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0
awful -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0
kwyjibo 0.5 -0.5 0.5 -0.5 0.5 -0.5 0.5 -0.5 0.5 -0.5 0.5 -0.5 0.5 -0.5 0.5 -0.5 0.5 -0.5 0.5 -0.5



> Bây giờ, cùng thiết lập các trường nào!

In [4]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                    tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

> Load dataset và tạo train, test, valid set.

In [5]:
from torchtext.legacy import datasets
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

> Lưu ý, ta chỉ có thể load custom embeddings sau khi "chúng" được chuyển thành `Vectors` object.

> Ta có thể tạo `Vector` object bằng cách truyền vào vị trí của embeddings (`name`), vị trí cho các cached embeddings (`cache`) và một hàm khởi tạo các tokens trong embeddings nhưng không xuất hiện bên trong dataset của chúng ta (`unk_init`). Tương tự các ví dụ trước, ta khởi tạo theo phân phối chuẩn tắc.

In [6]:
import torchtext.vocab as vocab

custom_embeddings = vocab.Vectors(name = 'custom_embeddings/embeddings.txt',
                                  cache = 'custom_embeddings',
                                  unk_init = torch.Tensor.normal_)

 86%|████████▌ | 6/7 [00:00<00:00, 402.61it/s]


> Để kiểm tra xem word embeddings có được load chính xác hay không, ta có thể hiển thị các words trong custom embeddings đó:

In [7]:
custom_embeddings.stoi

{'good': 0,
 'great': 1,
 'awesome': 2,
 'bad': 3,
 'terrible': 4,
 'awful': 5,
 'kwyjibo': 6}

> Ta cũng có thể trực tiếp in ra custom embeddings vector đó:

In [8]:
custom_embeddings.vectors

tensor([[ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
      

> Sau đó, ta sẽ tự xây dựng vocabulary và truyền vào đó `Vectors` object.

> Lưu ý là `unk_init` được khai báo khi tạo `Vectors`.

In [9]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = custom_embeddings)

LABEL.build_vocab(train_data)

> Bây giờ, ta cần kiểm tra xem vector từ điển cho các words trong custom embeddings phải khớp với nhau.

In [10]:
TEXT.vocab.vectors[TEXT.vocab.stoi['good']]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])

In [11]:
TEXT.vocab.vectors[TEXT.vocab.stoi['bad']]


tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1.])

> Các từ nằm trong custom embeddings nhưng không có trong dataset vocabulary được khởi tạo bởi hàm `unk_init`. Chúng có cùng size với custom embedding.

In [12]:
TEXT.vocab.vectors[TEXT.vocab.stoi['kwjibo']]


tensor([-0.1117, -0.4966,  0.1631, -0.8817,  0.2891,  0.4899, -0.3853, -0.7120,
         0.6369, -0.7141, -1.0831, -0.5547, -1.3248,  0.6970, -0.6631,  1.2158,
        -2.5273,  1.4778, -0.1696, -0.9919])

> Các phần phía sau tương tự như các notebooks trước.

In [13]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

> Ta cần đảm bảo `EMBEDDING_DIM` phải giống với embedding dimension trong custom embedding.

In [15]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 20
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 524,641 trainable parameters


> Tiếp theo, ta khởi tạo embedding layer để sử dụng các vocabulary vectors mà ta tạo trước đó.

In [17]:
embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.4778, -0.1696, -0.9919],
        [-0.5675, -0.2772, -2.1834,  ...,  0.8504,  1.0534,  0.3692],
        [-0.0552, -0.6125,  0.7500,  ..., -0.1261, -1.6770,  1.2068],
        ...,
        [ 0.5383, -0.1504,  1.6720,  ..., -0.3857, -1.0168,  0.1849],
        [ 2.5640, -0.8564, -0.0219,  ..., -0.3389,  0.2203, -1.6119],
        [ 0.1203,  1.5286,  0.6824,  ...,  0.3330, -0.6704,  0.5883]])

> Khởi tạo các tokens chưa biết và padding tokens embeddings là zero tensor.

In [18]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [19]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [20]:
criterion = nn.BCEWithLogitsLoss()


In [21]:
model = model.to(device)
criterion = criterion.to(device)


In [22]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc
    

In [23]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
            
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
                
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
    

In [25]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
    

### Freezing and Unfreezing Embeddings

> Chúng ta sẽ train model trong 10 epochs. Trong 5 epochs đầu, ta đóng bằng các weights của embedding layer. Trong 5 epochs cuối, ta sẽ mở khóa và train cả embedding layer.

> Câu hỏi đưa ra là: tại sao chúng ta phải làm như vậy? 
>> Câu trả lời là: Đôi khi, pre-trained word embeddings đã rất tốt rồi và không cần phải fine-tuned với model của chúng ta. Nếu đóng bằng embeddings pretrained, ta không cần phải tính gradients cũng như cập nhật weights cho các parameters này, dẫn đến tiết kiệm thời gian cho quá trình training. Một lý do thứ 2 là nếu train cả pre-trained word embeddings, số lượng weights là rất lớn dẫn đến việc khó train model.

> Để đóng băng embedding weights, ta set `model.ebedding.weight.requires_grad = False`. Câu lệnh này sẽ không tính đạo hàm cho các weights trong embedding layer, dẫn đến các parameters này không được cập nhật khi `optimizer.step()` được gọi.

> Nếu muốn train cả embedding layer, ta làm như sau: `model.ebedding.weight.requires_grad = True`.

In [26]:
N_EPOCHS = 10
FREEZE_FOR = 5

best_valid_loss = float('inf')

#freeze embeddings
model.embedding.weight.requires_grad = unfrozen = False

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s | Frozen? {not unfrozen}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tutC-model.pt')
    
    if (epoch + 1) >= FREEZE_FOR:
        #unfreeze embeddings
        model.embedding.weight.requires_grad = unfrozen = True

c:\Users\longln3\Anaconda3\envs\python3.7.6\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 01 | Epoch Time: 1m 59s | Frozen? True
	Train Loss: 0.727 | Train Acc: 52.50%
	 Val. Loss: 0.665 |  Val. Acc: 57.20%
Epoch: 02 | Epoch Time: 2m 5s | Frozen? True
	Train Loss: 0.663 | Train Acc: 60.26%
	 Val. Loss: 0.624 |  Val. Acc: 68.90%
Epoch: 03 | Epoch Time: 2m 56s | Frozen? True
	Train Loss: 0.635 | Train Acc: 63.42%
	 Val. Loss: 0.589 |  Val. Acc: 69.66%
Epoch: 04 | Epoch Time: 2m 14s | Frozen? True
	Train Loss: 0.608 | Train Acc: 66.04%
	 Val. Loss: 0.553 |  Val. Acc: 74.62%
Epoch: 05 | Epoch Time: 2m 17s | Frozen? True
	Train Loss: 0.587 | Train Acc: 68.52%
	 Val. Loss: 0.555 |  Val. Acc: 72.26%
Epoch: 06 | Epoch Time: 2m 49s | Frozen? False
	Train Loss: 0.566 | Train Acc: 70.77%
	 Val. Loss: 0.498 |  Val. Acc: 77.22%
Epoch: 07 | Epoch Time: 2m 44s | Frozen? False
	Train Loss: 0.520 | Train Acc: 74.30%
	 Val. Loss: 0.470 |  Val. Acc: 78.14%
Epoch: 08 | Epoch Time: 2m 15s | Frozen? False
	Train Loss: 0.485 | Train Acc: 76.66%
	 Val. Loss: 0.436 |  Val. Acc: 79.86%
Epoch:

In [ ]:
model.load_state_dict(torch.load('./models/tutC-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

### Saving Embeddings

> Đôi khi, ta muốn tái sử dụng các embeddings chúng ta đã train với model khác. Để thực hiện việc này, ta viết một hàm lặp xuyên suốt vocabulary, lấy từ và embedding của từ đó, ghi chúng vào text file sao cho đúng với format đã trình bày ở đầu notebooks.

In [30]:
from tqdm import tqdm

def write_embeddings(path, embeddings, vocab):
    
    with open(path, 'w') as f:
        for i, embedding in enumerate(tqdm(embeddings)):
            word = vocab.itos[i]
            #skip words with unicode symbols
            if len(word) != len(word.encode()):
                continue
            vector = ' '.join([str(i) for i in embedding.tolist()])
            f.write(f'{word} {vector}\n')

> Ghi embeddings vào file: 'custom_embeddings/trained_embeddings.txt'.

In [31]:
write_embeddings('custom_embeddings/trained_embeddings.txt', 
                 model.embedding.weight.data, 
                 TEXT.vocab)

100%|██████████| 25002/25002 [00:00<00:00, 31939.90it/s]


> Kiểm tra embeddings bằng cách load chúng vào vector.

In [32]:
trained_embeddings = vocab.Vectors(name = 'custom_embeddings/trained_embeddings.txt',
                                   cache = 'custom_embeddings',
                                   unk_init = torch.Tensor.normal_)

100%|█████████▉| 24941/24942 [00:00<00:00, 34337.03it/s]


> Cuối cùng, in cả 2 ra để so sánh.

In [33]:
print(trained_embeddings.vectors[:5])


tensor([[-0.1481, -0.2077,  0.2922, -0.1535,  0.0629, -0.0885, -0.2689, -0.2070,
         -0.1644,  0.0387,  0.1123, -0.1350,  0.1085, -0.1212, -0.1565, -0.0790,
         -0.1630,  0.1002, -0.2173, -0.0834],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2297, -0.4574,  0.8528, -0.8145, -0.1863,  0.0456, -1.5698, -0.2121,
          0.4783,  1.7190, -0.2239, -0.1312, -0.3100, -0.6320,  0.2988,  0.2503,
         -0.8606,  0.0651, -1.5308,  1.2659],
        [-0.5871, -0.0639,  0.2919, -0.6682, -0.4163, -0.4727, -1.5343,  0.8101,
          0.8359,  0.5311, -0.5475, -1.3023, -1.8893,  0.6650, -0.6499, -0.5865,
          0.3324, -0.4134,  0.6433,  0.8549],
        [-0.6817,  0.2149,  0.5038, -1.6466, -0.0353,  0.2784, -0.1395,  0.5631,
         -0.1565,  0.4011,  0.1388, -0.5444,  0.0086, -0.2714, -0.7601, -0.0708,
      

In [34]:
print(model.embedding.weight.data[:5])


tensor([[-0.1481, -0.2077,  0.2922, -0.1535,  0.0629, -0.0885, -0.2689, -0.2070,
         -0.1644,  0.0387,  0.1123, -0.1350,  0.1085, -0.1212, -0.1565, -0.0790,
         -0.1630,  0.1002, -0.2173, -0.0834],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2297, -0.4574,  0.8528, -0.8145, -0.1863,  0.0456, -1.5698, -0.2121,
          0.4783,  1.7190, -0.2239, -0.1312, -0.3100, -0.6320,  0.2988,  0.2503,
         -0.8606,  0.0651, -1.5308,  1.2659],
        [-0.5871, -0.0639,  0.2919, -0.6682, -0.4163, -0.4727, -1.5343,  0.8101,
          0.8359,  0.5311, -0.5475, -1.3023, -1.8893,  0.6650, -0.6499, -0.5865,
          0.3324, -0.4134,  0.6433,  0.8549],
        [-0.6817,  0.2149,  0.5038, -1.6466, -0.0353,  0.2784, -0.1395,  0.5631,
         -0.1565,  0.4011,  0.1388, -0.5444,  0.0086, -0.2714, -0.7601, -0.0708,
      

## END